In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
dataset=pd.read_csv('optical_multilayer_dataset.csv')

In [5]:
dataset.head()

,Refractive Indices,Thicknesses
0,"2.2061988652850064,1.85532877097426,1.06711037...","403.38465186122863,154.2439193420515,338.94717..."
1,"1.1868651217881563,2.474627177568472,1.9618022...","422.76554591423394,237.6485290560122,441.05361..."
2,"1.0178984775350002,1.0968955423415683,1.622609...","126.73288766527433,277.88067332442506,441.4921..."
3,"1.4514167888122065,2.084556677008474,2.3166845...","337.19612266709026,108.2681571851765,331.15334..."
4,"1.548006589181927,2.2442565772354097,1.8265288...","147.5510816999115,197.7501561619705,482.450832..."


In [6]:
def calculate_transfer_matrix(refractive_indices, thicknesses, wavelength):
    k = 2 * np.pi / wavelength
    num_layers = len(refractive_indices)
    M = np.eye(2, dtype=np.complex128) 
    for j in range(num_layers):
        nj = refractive_indices[j]
        dj = thicknesses[j]
        Mj = np.array([[np.cos(k * dj), -1j * np.sin(k * dj) / nj],
                       [-1j * nj * np.sin(k * dj), np.cos(k * dj)]], dtype=np.complex128)
        M = np.matmul(M, Mj)
    return M

In [7]:
def calculate_reflection(refractive_indices, thicknesses, wavelength, refractive_index_air, refractive_index_glass):
    n0 = refractive_index_air * np.sqrt(1.0 / (np.pi * 4e-7 * 8.854e-12))
    ns = refractive_index_glass * np.sqrt(1.0 / (np.pi * 4e-7 * 8.854e-12))
    R = []
    for i in range(len(refractive_indices)):
        refractive_indices_i = np.array(refractive_indices[i])
        thicknesses_i = np.array(thicknesses[i])
        M = calculate_transfer_matrix(refractive_indices_i, thicknesses_i, wavelength)
        M11, M12, M21, M22 = M[0, 0], M[0, 1], M[1, 0], M[1, 1]
        r = (M11 * n0 + M12 * n0 * ns - M21 - M22 * ns) / (M11 * n0 + M12 * n0 * ns + M21 + M22 * ns)
        Ri = np.abs(r) ** 2
        R.append(Ri)
    return R

In [8]:
wavelength = 600
refractive_index_air = 1.0
refractive_index_glass = 1.5
Reflectance = []

In [9]:
refractive_indices = []
thicknesses = []

In [10]:
for i in range(len(dataset)):
    refractive_indices_i = np.fromstring(dataset['Refractive Indices'][i], dtype=np.float64, sep=',')
    thicknesses_i = np.fromstring(dataset['Thicknesses'][i], dtype=np.float64, sep=',')
    refractive_indices.append(refractive_indices_i)
    thicknesses.append(thicknesses_i)

In [11]:
Reflectance = calculate_reflection(refractive_indices, thicknesses, wavelength, refractive_index_air, refractive_index_glass)

In [12]:
Reflectance

[0.9999999999999996,
 0.9999999999999998,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999999711,
 0.9999999999999998,
 0.9999999999999998,
 1.0,
 0.9999999999999998,
 0.9999999999999909,
 0.9999999999999998,
 1.0,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999986664,
 0.9999999999999998,
 0.9999999999999996,
 0.9999999999999718,
 1.0,
 0.9999999999999998,
 1.0,
 0.999999999999984,
 1.0,
 0.9999999999999996,
 1.0,
 0.9999999999999938,
 1.0,
 0.9999999999999933,
 0.9999999999999998,
 0.9999999999999993,
 1.0,
 0.9999999999999998,
 1.0,
 1.0,
 0.9999999999999996,
 1.0,
 0.9999999999999998,
 1.0,
 1.0,
 0.9999999999999996,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999998,
 0.9999999999996891,
 0.9999999999999925,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999998,
 0.9999999999999918,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9999999999999998,
 0.9999999999999998,
 1.0,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999999998,
 1.0,
 0.9999999999999996,
 0.9999999999999993,
 0.9999999999999996,
 0.9999999999999998,
 0.99

In [13]:
dataset['Reflectance'] = Reflectance

In [14]:
dataset

,Refractive Indices,Thicknesses,Reflectance
0,"2.2061988652850064,1.85532877097426,1.06711037...","403.38465186122863,154.2439193420515,338.94717...",1.0
1,"1.1868651217881563,2.474627177568472,1.9618022...","422.76554591423394,237.6485290560122,441.05361...",1.0
2,"1.0178984775350002,1.0968955423415683,1.622609...","126.73288766527433,277.88067332442506,441.4921...",1.0
3,"1.4514167888122065,2.084556677008474,2.3166845...","337.19612266709026,108.2681571851765,331.15334...",1.0
4,"1.548006589181927,2.2442565772354097,1.8265288...","147.5510816999115,197.7501561619705,482.450832...",1.0
...,...,...,...
9995,"1.8242541981223024,1.8260755310428323,1.911389...","272.93260645173376,279.0316840193202,399.31701...",1.0
9996,"1.4336120795398049,1.4773727341979705,1.531245...","123.23194350998001,418.7586087546599,149.15415...",1.0
9997,"2.489719901308742,1.292247136984523,2.01455631...","230.50306312895046,444.63734457375443,486.4412...",1.0
9998,"1.8047069917865224,1.1880161126727558,1.361080...","433.3402806861588,214.24265083707348,332.63526...",1.0


In [15]:
refractive_indices = dataset['Refractive Indices'].str.split(',', expand=True).astype(float)
refractive_indices.columns = ['RI_1', 'RI_2', 'RI_3', 'RI_4']

thicknesses = dataset['Thicknesses'].str.split(',', expand=True).astype(float)
thicknesses.columns = ['Thickness_1', 'Thickness_2', 'Thickness_3', 'Thickness_4']

data_split = pd.concat([thicknesses, refractive_indices, dataset['Reflectance']], axis=1)


In [16]:
data_split

,Thickness_1,Thickness_2,Thickness_3,Thickness_4,RI_1,RI_2,RI_3,RI_4,Reflectance
0,403.384652,154.243919,338.947175,226.374834,2.206199,1.855329,1.067110,1.775177,1.0
1,422.765546,237.648529,441.053615,282.223623,1.186865,2.474627,1.961802,2.296321,1.0
2,126.732888,277.880673,441.492114,137.058443,1.017898,1.096896,1.622609,2.308120,1.0
3,337.196123,108.268157,331.153343,483.897230,1.451417,2.084557,2.316685,1.079925,1.0
4,147.551082,197.750156,482.450832,430.079631,1.548007,2.244257,1.826529,1.114558,1.0
...,...,...,...,...,...,...,...,...,...
9995,272.932606,279.031684,399.317011,213.074628,1.824254,1.826076,1.911389,2.263532,1.0
9996,123.231944,418.758609,149.154151,175.563190,1.433612,1.477373,1.531245,1.771599,1.0
9997,230.503063,444.637345,486.441230,253.859991,2.489720,1.292247,2.014556,1.823121,1.0
9998,433.340281,214.242651,332.635262,455.765651,1.804707,1.188016,1.361081,2.428262,1.0


In [17]:
X =data_split[['Thickness_1', 'Thickness_2','Thickness_3', 'Thickness_4','Reflectance']]
y = data_split[['RI_1','RI_2','RI_3','RI_4']]

In [18]:
X

,Thickness_1,Thickness_2,Thickness_3,Thickness_4,Reflectance
0,403.384652,154.243919,338.947175,226.374834,1.0
1,422.765546,237.648529,441.053615,282.223623,1.0
2,126.732888,277.880673,441.492114,137.058443,1.0
3,337.196123,108.268157,331.153343,483.897230,1.0
4,147.551082,197.750156,482.450832,430.079631,1.0
...,...,...,...,...,...
9995,272.932606,279.031684,399.317011,213.074628,1.0
9996,123.231944,418.758609,149.154151,175.563190,1.0
9997,230.503063,444.637345,486.441230,253.859991,1.0
9998,433.340281,214.242651,332.635262,455.765651,1.0


In [19]:
y

,RI_1,RI_2,RI_3,RI_4
0,2.206199,1.855329,1.067110,1.775177
1,1.186865,2.474627,1.961802,2.296321
2,1.017898,1.096896,1.622609,2.308120
3,1.451417,2.084557,2.316685,1.079925
4,1.548007,2.244257,1.826529,1.114558
...,...,...,...,...
9995,1.824254,1.826076,1.911389,2.263532
9996,1.433612,1.477373,1.531245,1.771599
9997,2.489720,1.292247,2.014556,1.823121
9998,1.804707,1.188016,1.361081,2.428262


In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1])) 

In [22]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [23]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 71.1595
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5228
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 1.0446
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.8309
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.7398
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6587
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6173
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6497
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5412
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5916
Epoch 11/100
250/250 [==============================] - 0s 1ms/step - loss: 0.6055
Epoch 12/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5723
Epoch 13/100

250/250 [==============================] - 0s 1ms/step - loss: 0.2320
Epoch 100/100
250/250 [==============================] - 0s 1ms/step - loss: 0.2365


In [24]:
predictions = model.predict(X_test)
predictions

63/63 [==============================] - 0s 4ms/step


array([[1.5726929, 1.402732 , 2.1289232, 1.637486 ],
       [1.7830522, 1.4476696, 2.0704925, 1.7376494],
       [1.3903756, 1.5105957, 1.9285576, 1.678913 ],
       ...,
       [1.6334867, 1.4940878, 1.8114636, 1.5570943],
       [1.5825636, 1.6197006, 1.9328651, 1.7075145],
       [1.5394679, 1.5399626, 1.9635413, 1.5901594]], dtype=float32)

In [25]:
loss = model.evaluate(X_test, y_test)
loss

63/63 [==============================] - 0s 1ms/step - loss: 0.2270


0.226997971534729

In [1]:
a=calculate_reflection(refractive_indices, thicknesses, wavelength, refractive_index_air, refractive_index_glass)

NameError: name 'calculate_transfer_matrix' is not defined

In [2]:
refractive_indices

NameError: name 'refractive_indices' is not defined